# UFC Stats – Fighter Analysis

**Change the two fighter names below to compare any pair.** All data is fetched from the [UFC Stats API](https://ufcapi.aristotle.me).

In [1]:
# ============================================================
# CHANGE THESE TWO FIGHTER NAMES TO ANALYZE ANY PAIR
# ============================================================
FIGHTER_1 = "Charles Oliveira"
FIGHTER_2 = "Max Holloway"

In [2]:
import sys
from pathlib import Path

import pandas as pd
import plotly.graph_objects as go
from IPython.display import HTML, display

# Show all DataFrame columns (no truncation)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

# Enable text wrapping in DataFrame cells
display(HTML("""
<style>
  .dataframe td, .dataframe th {
    white-space: normal !important;
    word-wrap: break-word;
    max-width: 400px;
  }
</style>
"""))

# Add project root (fight-lab) so we can import src
root = Path.cwd()
if not (root / "src" / "ufc_stats").exists():
    root = root.parent  # running from notebooks/
sys.path.insert(0, str(root))

from src.ufc_stats import UFCStatsClient
from src.standard_metrics import (
    record_comparison_figure,
    career_stats_comparison_figure,
    cumulative_wins_figure,
    streak_and_last_5,
    head_to_head_advantages,
    strikes_absorbed_by_target_figures,
    strikes_by_position_figures,
    finishes_by_round_method_figures,
    takedowns_per_fight_figure,
    takedowns_absorbed_per_round_figures,
)

client = UFCStatsClient()

In [3]:
# Fetch all data for both fighters (includes normalized my_*/opp_* columns)
data = client.get_all_data_for_fighters(FIGHTER_1, FIGHTER_2)

# Overview

High-level summary charts for both fighters.

## Record / win-loss comparison

In [4]:
fig = record_comparison_figure(data, FIGHTER_1, FIGHTER_2)
fig.show()

## Career stats comparison

In [5]:
fig = career_stats_comparison_figure(data, FIGHTER_1, FIGHTER_2)
if fig is not None:
    fig.show()

## Win/loss trend over fights

In [6]:
_, _, fig = cumulative_wins_figure(data, FIGHTER_1, FIGHTER_2)
fig.show()

## Streak / momentum summary

In [7]:
streak_data = streak_and_last_5(data, FIGHTER_1, FIGHTER_2)
for name in [FIGHTER_1, FIGHTER_2]:
    info = streak_data[name]
    print(f"**{name}**: Current streak = {info['streak']:+d}")
    print("Last 5 fights (most recent first):")
    display(info["last_5"])

**Charles Oliveira**: Current streak = +1
Last 5 fights (most recent first):


,opponent,result,method,event_date
0,Mateusz Gamrot,win,SUB Rear Naked Choke,"Oct. 11, 2025"
1,Ilia Topuria,loss,KO/TKO Punch,"Jun. 28, 2025"
2,Michael Chandler,win,U-DEC,"Nov. 16, 2024"
3,Arman Tsarukyan,loss,S-DEC,"Apr. 13, 2024"
4,Beneil Dariush,win,KO/TKO Punches,"Jun. 10, 2023"


**Max Holloway**: Current streak = +1
Last 5 fights (most recent first):


,opponent,result,method,event_date
0,Dustin Poirier,win,U-DEC,"Jul. 19, 2025"
1,Ilia Topuria,loss,KO/TKO Punch,"Oct. 26, 2024"
2,Justin Gaethje,win,KO/TKO Punch,"Apr. 13, 2024"
3,Chan Sung Jung,win,KO/TKO Punch,"Aug. 26, 2023"
4,Arnold Allen,win,U-DEC,"Apr. 15, 2023"


## Head-to-head advantages

In [8]:
adv = head_to_head_advantages(data, FIGHTER_1, FIGHTER_2)
for name in [FIGHTER_1, FIGHTER_2]:
    val = adv.get(name)
    if val:
        print(f"**{name}** leads in: {val}")

**Charles Oliveira** leads in: Strike Accuracy, Strikes Absorbed/Min, Takedown Avg, Submission Avg
**Max Holloway** leads in: Strikes Landed/Min, Strike Defense, Takedown Accuracy, Takedown Defense


# Secondary Breakdowns

Striking and grappling analysis with opponent context. Each point/bar shows who the opponent was on hover.

## Striking

### Strikes absorbed by target (defense profile)

Per-round averages. Blue bars: what each fighter absorbs. Red bars (below 0): what their opponent typically lands.

In [9]:
for fig in strikes_absorbed_by_target_figures(data, FIGHTER_1, FIGHTER_2):
    fig.show()

### Per-fighter striking profile

Strikes absorbed and opponent landing distribution for each fighter, with counts and percentages.

In [10]:
# Per-fighter striking profile is shown above (absorbed vs opponent lands, per round)

### Strikes by position: distance / clinch / ground

Where each fighter lands (offense) and absorbs (defense). Each bar = one fight; hover for opponent.

In [11]:
for fig in strikes_by_position_figures(data, FIGHTER_1, FIGHTER_2):
    fig.show()

### Finishes (by round and method)

KO/TKO and striking-based finishes. Hover for opponent.

In [12]:
for fig in finishes_by_round_method_figures(data, FIGHTER_1, FIGHTER_2):
    fig.show()

## Grappling

### Takedowns and control time

Takedowns landed vs absorbed per fight. Hover for opponent.

In [13]:
fig = takedowns_per_fight_figure(data, FIGHTER_1, FIGHTER_2)
if fig is not None:
    fig.show()

### Per-fighter grappling profile

Takedowns per round. Blue: absorbed/landed. Red (below 0): opponent’s inverse.

In [14]:
for fig in takedowns_absorbed_per_round_figures(data, FIGHTER_1, FIGHTER_2):
    fig.show()

## Add your own analysis

In [15]:
# Add your own analysis below